# TASK 1:

In [6]:
library(httr)
library(jsonlite)

url_coinmarcap <- "https://api.coinmarketcap.com"
path_ticker <- "v1/ticker/"

raw.data <- GET(url = url_coinmarcap, path = path_ticker)
raw.content <- rawToChar(raw.data$content)

neat_content <- fromJSON(raw.content)

head(neat_content)


id,name,symbol,rank,price_usd,price_btc,24h_volume_usd,market_cap_usd,available_supply,total_supply,max_supply,percent_change_1h,percent_change_24h,percent_change_7d,last_updated
bitcoin,Bitcoin,BTC,1,11168.9,1.0,6257030000.0,188789033590,16903100.0,16903100.0,21000000.0,-1.44,-2.63,5.08,1520331865
ethereum,Ethereum,ETH,2,840.876,0.0754461,1866220000.0,82419487376.0,98016220.0,98016220.0,NA,-0.91,-2.64,-4.87,1520331851
ripple,Ripple,XRP,3,0.954691,0.00008566,1172350000.0,37320739240.0,39091956706.0,99992520283.0,100000000000,-2.85,-9.42,-0.44,1520331840
bitcoin-cash,Bitcoin Cash,BCH,4,1237.72,0.111052,362546000.0,21044535430.0,17002663.0,17002663.0,21000000.0,-1.14,-3.35,-2.64,1520331852
litecoin,Litecoin,LTC,5,205.994,0.0184825,592015000.0,11433704490.0,55505037.0,55505037.0,84000000.0,-0.68,-2.89,-6.21,1520331840
cardano,Cardano,ADA,6,0.294036,0.00002638,206738000.0,7623492113.0,25927070538.0,31112483745.0,45000000000.0,-2.28,-1.1,-12.68,1520331854
